In [1]:
import pandas as pd
import numpy as np
from itertools import product,chain
import pandas_profiling as pp

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import matthews_corrcoef
from scipy.stats import rankdata


import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import catboost as cb
from sklearn.pipeline import Pipeline
import lightgbm as lgb




In [2]:
train = pd.read_csv('train.csv', sep = '\t', index_col = 'Unnamed: 0')
test = pd.read_csv('test.csv', sep = '\t', index_col = 'Unnamed: 0')

train.drop(['140', '152', '160', '164','305', '303', '301', '193', '192', '186', '150', '141', '138', '130'],inplace = True,axis = 1)
test.drop(['140', '152', '160', '164','305', '303', '301', '193', '192', '186', '150', '141', '138', '130'],inplace = True, axis = 1)


X_train, y_train = train[train.columns[1:]], train['0']
X_test, y_test = test[test.columns[1:]], test['0'].fillna(0, inplace = True)

In [5]:
skf = StratifiedKFold(n_splits=4, random_state=42)
# для баланса классов в некоторых моделях 
spw = (len(y_train) - np.sum(y_train))/np.sum(y_train)

# для катбуста выделю категориальные переменные 
category_cols = [i for i in X_train.columns if len(X_train[i].value_counts()) == 2 ]
cat_dims = [X_train.columns.get_loc(i) for i in category_cols[:-1]] 

xgb_train = xgb.DMatrix(X_train, y_train, feature_names=X_train.columns)
xgb_test = xgb.DMatrix(X_test, feature_names=X_test.columns)

In [6]:
def check_train_score(params, lgb_data, data, target, kf, num_rounds):
    roc_auc = []
    for train, val in kf.split(data,target):
        temp_lgb_train = lgb_data.subset(train)
        temp_lgb_val = lgb_data.subset(val)
        temp_model = lgb.train(params, temp_lgb_train, num_rounds, verbose_eval=num_rounds)
        roc_auc.append([roc_auc_score(target[train],temp_model.predict(data.loc[train])), roc_auc_score(target[val],temp_model.predict(data.loc[val]))])
    return np.mean(roc_auc, axis=0)

# XGBOOST


In [10]:
parameters = {
    #default
    'objective': 'reg:logistic',
    'eta': 0.01,
    'silent': 1,
    "nthread": 4,
    "random_seed": 1,
    "eval_metric": 'auc',
    'max_depth':7,
    #'scale_pos_weight':spw,
    
    'booster':'dart',
    'rate_drop':0.3,
    'skip_drop':0.4,
    
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    
    'tree_method': 'hist',
    'grow_policy': 'lossguide'
    #'min_child_weight':5

}




In [11]:
results = xgb.cv(parameters, xgb_train, num_boost_round=2000, early_stopping_rounds = 250,folds=skf.split(X_train, y_train), verbose_eval=1)

[0]	train-auc:0.641584+0.100167	test-auc:0.617154+0.0831211
[1]	train-auc:0.73749+0.00249814	test-auc:0.629113+0.0913316
[2]	train-auc:0.746458+0.00655328	test-auc:0.691738+0.00319867
[3]	train-auc:0.747914+0.00694292	test-auc:0.686254+0.010242
[4]	train-auc:0.752929+0.00379832	test-auc:0.69877+0.00379895
[5]	train-auc:0.75687+0.00438163	test-auc:0.701887+0.00530887
[6]	train-auc:0.755796+0.00750246	test-auc:0.704504+0.00547891
[7]	train-auc:0.765702+0.00322196	test-auc:0.705714+0.00651177
[8]	train-auc:0.761315+0.00475864	test-auc:0.702217+0.00659699
[9]	train-auc:0.767557+0.00260603	test-auc:0.704756+0.00378408
[10]	train-auc:0.769489+0.00141916	test-auc:0.706336+0.00606448
[11]	train-auc:0.771123+0.00266977	test-auc:0.708387+0.00475278
[12]	train-auc:0.771642+0.00216663	test-auc:0.710064+0.00514607
[13]	train-auc:0.766576+0.00191866	test-auc:0.707784+0.00742002
[14]	train-auc:0.773998+0.00430377	test-auc:0.70971+0.00486978
[15]	train-auc:0.774101+0.00420819	test-auc:0.707128+0.00569

[129]	train-auc:0.797613+0.00271015	test-auc:0.721633+0.00500294
[130]	train-auc:0.79933+0.00222819	test-auc:0.72184+0.00500673
[131]	train-auc:0.798956+0.00158364	test-auc:0.721877+0.00530614
[132]	train-auc:0.799015+0.00233732	test-auc:0.721336+0.00472925
[133]	train-auc:0.797736+0.00243625	test-auc:0.721907+0.00479935
[134]	train-auc:0.799274+0.00223076	test-auc:0.721076+0.00433662
[135]	train-auc:0.799333+0.00302752	test-auc:0.721666+0.00454665
[136]	train-auc:0.798583+0.00287831	test-auc:0.721981+0.00476668
[137]	train-auc:0.799426+0.00174083	test-auc:0.722318+0.00504501
[138]	train-auc:0.800376+0.00281244	test-auc:0.722662+0.00528084
[139]	train-auc:0.799469+0.00277142	test-auc:0.721944+0.00482014
[140]	train-auc:0.799746+0.00340393	test-auc:0.722519+0.00514073
[141]	train-auc:0.80051+0.00350632	test-auc:0.722325+0.00500916
[142]	train-auc:0.800477+0.00263742	test-auc:0.722527+0.00502511
[143]	train-auc:0.801058+0.00291895	test-auc:0.72193+0.00460658
[144]	train-auc:0.799477+0.00

[256]	train-auc:0.815825+0.00375084	test-auc:0.72603+0.00495308
[257]	train-auc:0.814774+0.00398858	test-auc:0.725785+0.00499866
[258]	train-auc:0.816557+0.00272385	test-auc:0.725981+0.00502079
[259]	train-auc:0.816377+0.00223655	test-auc:0.725598+0.00525031
[260]	train-auc:0.817624+0.00281937	test-auc:0.725733+0.00465651
[261]	train-auc:0.816829+0.00265667	test-auc:0.72577+0.0050031
[262]	train-auc:0.81717+0.00300269	test-auc:0.725766+0.00473399
[263]	train-auc:0.816228+0.0045773	test-auc:0.725532+0.00519632
[264]	train-auc:0.816204+0.00305254	test-auc:0.726128+0.00502737
[265]	train-auc:0.817408+0.00220402	test-auc:0.725693+0.00476616
[266]	train-auc:0.816804+0.00479104	test-auc:0.726116+0.00526507
[267]	train-auc:0.816256+0.00300273	test-auc:0.726111+0.00496476
[268]	train-auc:0.81741+0.00157217	test-auc:0.72567+0.00477355
[269]	train-auc:0.818316+0.00441526	test-auc:0.725653+0.0046936
[270]	train-auc:0.816773+0.00410681	test-auc:0.726068+0.0053503
[271]	train-auc:0.817535+0.0021379

[383]	train-auc:0.831293+0.00130489	test-auc:0.729516+0.00323502
[384]	train-auc:0.831961+0.00167078	test-auc:0.729513+0.00331766
[385]	train-auc:0.832666+0.00208671	test-auc:0.72952+0.0033041
[386]	train-auc:0.831521+0.00202183	test-auc:0.729912+0.00348769
[387]	train-auc:0.832623+0.00244994	test-auc:0.72933+0.0032263
[388]	train-auc:0.83142+0.00131905	test-auc:0.729675+0.00337399
[389]	train-auc:0.83155+0.00239034	test-auc:0.729774+0.00345539
[390]	train-auc:0.831868+0.00333162	test-auc:0.729512+0.00313734
[391]	train-auc:0.833906+0.00309916	test-auc:0.729544+0.00314681
[392]	train-auc:0.832291+0.00344808	test-auc:0.729537+0.00362768
[393]	train-auc:0.833619+0.00234745	test-auc:0.7293+0.00309213
[394]	train-auc:0.834683+0.00232314	test-auc:0.729806+0.00330362
[395]	train-auc:0.834279+0.00330567	test-auc:0.730046+0.00336229
[396]	train-auc:0.834146+0.00222595	test-auc:0.730282+0.00252436
[397]	train-auc:0.834713+0.00249052	test-auc:0.730005+0.00332053
[398]	train-auc:0.833975+0.001852

[510]	train-auc:0.846548+0.000713689	test-auc:0.733029+0.00228164
[511]	train-auc:0.845586+0.00215769	test-auc:0.73334+0.00207109
[512]	train-auc:0.848019+0.000534391	test-auc:0.733402+0.00207662
[513]	train-auc:0.847247+0.000814457	test-auc:0.732994+0.00270284
[514]	train-auc:0.84698+0.00158782	test-auc:0.733497+0.00183892
[515]	train-auc:0.847438+0.00123186	test-auc:0.733562+0.00142273
[516]	train-auc:0.849317+0.00176856	test-auc:0.733343+0.00202974
[517]	train-auc:0.848748+0.000189373	test-auc:0.732716+0.00158202
[518]	train-auc:0.848472+0.000661018	test-auc:0.733606+0.00189434
[519]	train-auc:0.848465+0.000669263	test-auc:0.73346+0.00210499
[520]	train-auc:0.848187+0.00275328	test-auc:0.732862+0.00177785
[521]	train-auc:0.849293+0.000976811	test-auc:0.733121+0.0018647
[522]	train-auc:0.847793+0.00149498	test-auc:0.733304+0.00194139
[523]	train-auc:0.848617+0.000736967	test-auc:0.733449+0.00198341
[524]	train-auc:0.849021+0.000567035	test-auc:0.733171+0.0012384
[525]	train-auc:0.849

...

[1386]	train-auc:0.911345+0.00324571	test-auc:0.741285+0.00130396
[1387]	train-auc:0.910627+0.00160643	test-auc:0.74057+0.00169047
[1388]	train-auc:0.911453+0.000339471	test-auc:0.741112+0.00116695
[1389]	train-auc:0.910127+0.00175286	test-auc:0.741306+0.00131608
[1390]	train-auc:0.911412+0.00130493	test-auc:0.741588+0.00126767
[1391]	train-auc:0.911199+0.000832457	test-auc:0.741124+0.00083307
[1392]	train-auc:0.911518+0.000985746	test-auc:0.741035+0.000872728
[1393]	train-auc:0.912321+0.00070379	test-auc:0.740969+0.00105975
[1394]	train-auc:0.911095+0.00205938	test-auc:0.741085+0.00111229
[1395]	train-auc:0.913855+0.00128272	test-auc:0.741257+0.00127822
[1396]	train-auc:0.910914+0.0019282	test-auc:0.740754+0.00120878
[1397]	train-auc:0.911323+0.00152277	test-auc:0.741272+0.00125289
[1398]	train-auc:0.912822+0.00133061	test-auc:0.741232+0.00115781
[1399]	train-auc:0.913828+0.000859126	test-auc:0.740057+0.00157046
[1400]	train-auc:0.913375+0.000100959	test-auc:0.740818+0.00106763
[1401]

[1510]	train-auc:0.915994+0.00256119	test-auc:0.74162+0.00107835
[1511]	train-auc:0.91694+0.00141886	test-auc:0.741633+0.00105884
[1512]	train-auc:0.918617+0.00106764	test-auc:0.741803+0.00111307
[1513]	train-auc:0.917605+0.000774738	test-auc:0.7413+0.00118279
[1514]	train-auc:0.919675+0.00109709	test-auc:0.740971+0.000619583
[1515]	train-auc:0.918603+0.000816697	test-auc:0.741502+0.00109309
[1516]	train-auc:0.918244+0.000375384	test-auc:0.74109+0.00106928
[1517]	train-auc:0.91796+0.000878237	test-auc:0.741338+0.00151396
[1518]	train-auc:0.918228+0.000941737	test-auc:0.741537+0.00129214
[1519]	train-auc:0.918729+0.000831986	test-auc:0.741166+0.00086566
[1520]	train-auc:0.91809+0.0012671	test-auc:0.741492+0.00127117
[1521]	train-auc:0.917188+0.000898433	test-auc:0.741152+0.00187312
[1522]	train-auc:0.919113+0.00084145	test-auc:0.74165+0.000978102
[1523]	train-auc:0.92069+0.00142005	test-auc:0.741612+0.000922811
[1524]	train-auc:0.918117+0.0012703	test-auc:0.741584+0.000902708
[1525]	tra

[1634]	train-auc:0.922867+0.00160805	test-auc:0.741455+0.000915354
[1635]	train-auc:0.925052+0.00102802	test-auc:0.741296+0.00148387
[1636]	train-auc:0.923967+0.000239898	test-auc:0.741487+0.00106441
[1637]	train-auc:0.925436+0.00150732	test-auc:0.741298+0.00117331
[1638]	train-auc:0.924223+0.00215531	test-auc:0.740564+0.00163397
[1639]	train-auc:0.923392+0.00209499	test-auc:0.741247+0.000669378
[1640]	train-auc:0.925284+0.00132106	test-auc:0.741155+0.00119229
[1641]	train-auc:0.924119+0.000420441	test-auc:0.741595+0.00094433
[1642]	train-auc:0.924199+0.000597132	test-auc:0.741433+0.000791082
[1643]	train-auc:0.92352+0.00148551	test-auc:0.740822+0.00157461
[1644]	train-auc:0.924302+0.000206961	test-auc:0.74124+0.00153918
[1645]	train-auc:0.922901+0.0020462	test-auc:0.740723+0.00121545
[1646]	train-auc:0.923975+0.00105415	test-auc:0.741886+0.00112087
[1647]	train-auc:0.922571+0.00145268	test-auc:0.741044+0.000782498
[1648]	train-auc:0.925401+0.00113196	test-auc:0.740736+0.000991501
[164

[1758]	train-auc:0.929507+0.00125238	test-auc:0.741226+0.00174121
[1759]	train-auc:0.929296+0.000578189	test-auc:0.742035+0.0009005
[1760]	train-auc:0.928033+0.00224472	test-auc:0.741844+0.00133009
[1761]	train-auc:0.928183+0.00202811	test-auc:0.741302+0.00132155
[1762]	train-auc:0.92933+0.000574675	test-auc:0.741437+0.00161194
[1763]	train-auc:0.929414+0.000543329	test-auc:0.74184+0.00115177
[1764]	train-auc:0.92945+0.00252083	test-auc:0.741542+0.00149262
[1765]	train-auc:0.928592+0.00153844	test-auc:0.741664+0.000974694
[1766]	train-auc:0.930137+0.00179797	test-auc:0.741338+0.000824337
[1767]	train-auc:0.92902+0.00115492	test-auc:0.741792+0.0016194
[1768]	train-auc:0.930521+0.00105522	test-auc:0.741923+0.00132367
[1769]	train-auc:0.930977+0.00159416	test-auc:0.742219+0.00146793
[1770]	train-auc:0.929026+0.00195571	test-auc:0.741578+0.00112989
[1771]	train-auc:0.928938+0.00156868	test-auc:0.741064+0.00107796
[1772]	train-auc:0.930702+0.00221139	test-auc:0.741241+0.00166603
[1773]	trai

[1882]	train-auc:0.935487+0.00156442	test-auc:0.740955+0.00203092
[1883]	train-auc:0.933794+0.000251166	test-auc:0.741036+0.00214761
[1884]	train-auc:0.932984+0.000193419	test-auc:0.741781+0.00187861
[1885]	train-auc:0.93412+0.000923849	test-auc:0.741942+0.00168248
[1886]	train-auc:0.933873+0.000539712	test-auc:0.74171+0.0014433
[1887]	train-auc:0.934112+0.00157771	test-auc:0.741617+0.00180349
[1888]	train-auc:0.934562+0.00132064	test-auc:0.74116+0.00186942
[1889]	train-auc:0.933936+0.00149465	test-auc:0.741818+0.00165015
[1890]	train-auc:0.934969+0.000217878	test-auc:0.742222+0.00151503
[1891]	train-auc:0.933138+0.00181237	test-auc:0.74159+0.00159346
[1892]	train-auc:0.934582+0.000455904	test-auc:0.741158+0.00194702
[1893]	train-auc:0.93562+0.00248623	test-auc:0.741729+0.00144555
[1894]	train-auc:0.934594+0.000385864	test-auc:0.741715+0.00164906
[1895]	train-auc:0.934617+0.000356349	test-auc:0.741175+0.00261301
[1896]	train-auc:0.935242+0.000886888	test-auc:0.741819+0.00147045
[1897]	

In [12]:
np.max(results['test-auc-mean']), np.argmax(results['test-auc-mean'])

/Users/mefkov/miniconda3/envs/py3/lib/python3.5/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


(0.7423846666666667, 1956)

In [13]:
model = xgb.train(parameters, xgb_train, num_boost_round = 1956, verbose_eval=4)


In [14]:
y_pred_last = model.predict(xgb_test)

In [20]:
test['0'] = y_pred_last
header = ['_VAL_']
test['0'].to_csv('xgb4.csv',sep =',', header = header, index_label = ['_ID_']) 
# LB 0.760809
# CV 0.7423846

# LGBM

In [ ]:
lgb_train = lgb.Dataset(X_train, label=y_train, free_raw_data=False)
lgb_test = lgb.Dataset(X_test,free_raw_data=False)

In [33]:
parameters = {
    'objective': 'binary',
    'learning_rate': 0.01,
    'num_threads': 4,
    "metric": 'auc', 
       
    'colsample_bytree': 0.7,
    'subsample': 0.7,
    'subsample_freq': 1,
    'min_data_in_leaf': 10
}
n_rounds = 2500




In [34]:
result = lgb.cv(parameters, lgb_train, n_rounds, folds=skf.split(X_train, y_train), early_stopping_rounds=250, verbose_eval=1, )

[1]	cv_agg's auc: 0.680263 + 0.00698339
[2]	cv_agg's auc: 0.693429 + 0.00589809
[3]	cv_agg's auc: 0.697974 + 0.00513605
[4]	cv_agg's auc: 0.70106 + 0.00593674
[5]	cv_agg's auc: 0.702868 + 0.00515088
[6]	cv_agg's auc: 0.704958 + 0.00517652
[7]	cv_agg's auc: 0.708008 + 0.00482082
[8]	cv_agg's auc: 0.708519 + 0.0049205
[9]	cv_agg's auc: 0.708963 + 0.0045892
[10]	cv_agg's auc: 0.709558 + 0.00430383
[11]	cv_agg's auc: 0.709449 + 0.00429489
[12]	cv_agg's auc: 0.709561 + 0.00422176
[13]	cv_agg's auc: 0.71089 + 0.00365398
[14]	cv_agg's auc: 0.710881 + 0.00349799
[15]	cv_agg's auc: 0.711048 + 0.00377237
[16]	cv_agg's auc: 0.711606 + 0.0034799
[17]	cv_agg's auc: 0.711595 + 0.00350715
[18]	cv_agg's auc: 0.711873 + 0.0028211
[19]	cv_agg's auc: 0.713068 + 0.00299589
[20]	cv_agg's auc: 0.71344 + 0.00328171
[21]	cv_agg's auc: 0.714113 + 0.00323806
[22]	cv_agg's auc: 0.714909 + 0.00309827
[23]	cv_agg's auc: 0.715208 + 0.00307809
[24]	cv_agg's auc: 0.715167 + 0.00313916
[25]	cv_agg's auc: 0.714963 + 0.

[200]	cv_agg's auc: 0.731798 + 0.00425369
[201]	cv_agg's auc: 0.73185 + 0.00421029
[202]	cv_agg's auc: 0.731998 + 0.00413402
[203]	cv_agg's auc: 0.731947 + 0.004186
[204]	cv_agg's auc: 0.731977 + 0.00419052
[205]	cv_agg's auc: 0.732055 + 0.00424759
[206]	cv_agg's auc: 0.732114 + 0.00425718
[207]	cv_agg's auc: 0.732174 + 0.00424934
[208]	cv_agg's auc: 0.732262 + 0.00417269
[209]	cv_agg's auc: 0.732347 + 0.00420802
[210]	cv_agg's auc: 0.732351 + 0.00428108
[211]	cv_agg's auc: 0.732402 + 0.00429291
[212]	cv_agg's auc: 0.732498 + 0.00427694
[213]	cv_agg's auc: 0.732602 + 0.00432869
[214]	cv_agg's auc: 0.732691 + 0.00434653
[215]	cv_agg's auc: 0.732768 + 0.00435031
[216]	cv_agg's auc: 0.732899 + 0.00430722
[217]	cv_agg's auc: 0.733005 + 0.0043227
[218]	cv_agg's auc: 0.733096 + 0.0043607
[219]	cv_agg's auc: 0.733179 + 0.00437036
[220]	cv_agg's auc: 0.73323 + 0.00438059
[221]	cv_agg's auc: 0.733271 + 0.00442974
[222]	cv_agg's auc: 0.733348 + 0.00441459
[223]	cv_agg's auc: 0.733446 + 0.0045093

[397]	cv_agg's auc: 0.741432 + 0.00456322
[398]	cv_agg's auc: 0.741478 + 0.00457136
[399]	cv_agg's auc: 0.741486 + 0.00460453
[400]	cv_agg's auc: 0.741503 + 0.00462857
[401]	cv_agg's auc: 0.741547 + 0.00468206
[402]	cv_agg's auc: 0.741591 + 0.00471362
[403]	cv_agg's auc: 0.741556 + 0.00474771
[404]	cv_agg's auc: 0.741629 + 0.00475218
[405]	cv_agg's auc: 0.74166 + 0.00472042
[406]	cv_agg's auc: 0.741652 + 0.00470363
[407]	cv_agg's auc: 0.741709 + 0.00468591
[408]	cv_agg's auc: 0.741757 + 0.00472354
[409]	cv_agg's auc: 0.741786 + 0.00474647
[410]	cv_agg's auc: 0.741849 + 0.00474656
[411]	cv_agg's auc: 0.741866 + 0.0047409
[412]	cv_agg's auc: 0.741897 + 0.00470757
[413]	cv_agg's auc: 0.741903 + 0.00470293
[414]	cv_agg's auc: 0.74197 + 0.00474515
[415]	cv_agg's auc: 0.742003 + 0.00476348
[416]	cv_agg's auc: 0.742018 + 0.00474193
[417]	cv_agg's auc: 0.741994 + 0.00472527
[418]	cv_agg's auc: 0.742015 + 0.0047107
[419]	cv_agg's auc: 0.742047 + 0.00471653
[420]	cv_agg's auc: 0.742131 + 0.00471

[594]	cv_agg's auc: 0.744581 + 0.00531934
[595]	cv_agg's auc: 0.744638 + 0.00529519
[596]	cv_agg's auc: 0.744655 + 0.00528476
[597]	cv_agg's auc: 0.744629 + 0.00529018
[598]	cv_agg's auc: 0.744628 + 0.00530598
[599]	cv_agg's auc: 0.744623 + 0.00530366
[600]	cv_agg's auc: 0.74465 + 0.00530474
[601]	cv_agg's auc: 0.744635 + 0.00533085
[602]	cv_agg's auc: 0.744659 + 0.0053485
[603]	cv_agg's auc: 0.744689 + 0.00537221
[604]	cv_agg's auc: 0.744711 + 0.00536823
[605]	cv_agg's auc: 0.744705 + 0.00536914
[606]	cv_agg's auc: 0.744701 + 0.0053543
[607]	cv_agg's auc: 0.744701 + 0.00537447
[608]	cv_agg's auc: 0.744689 + 0.00538257
[609]	cv_agg's auc: 0.744709 + 0.00540385
[610]	cv_agg's auc: 0.744676 + 0.00542349
[611]	cv_agg's auc: 0.744699 + 0.00539512
[612]	cv_agg's auc: 0.744736 + 0.00539175
[613]	cv_agg's auc: 0.744766 + 0.00544251
[614]	cv_agg's auc: 0.744764 + 0.00543499
[615]	cv_agg's auc: 0.744809 + 0.00542495
[616]	cv_agg's auc: 0.74482 + 0.00541621
[617]	cv_agg's auc: 0.744849 + 0.00543

[792]	cv_agg's auc: 0.745525 + 0.0055346
[793]	cv_agg's auc: 0.745548 + 0.00551886
[794]	cv_agg's auc: 0.745546 + 0.005541
[795]	cv_agg's auc: 0.745533 + 0.00557575
[796]	cv_agg's auc: 0.745541 + 0.00556114
[797]	cv_agg's auc: 0.745579 + 0.00554489
[798]	cv_agg's auc: 0.745588 + 0.00555381
[799]	cv_agg's auc: 0.745587 + 0.00554136
[800]	cv_agg's auc: 0.745589 + 0.0055572
[801]	cv_agg's auc: 0.745579 + 0.00559872
[802]	cv_agg's auc: 0.745579 + 0.00555948
[803]	cv_agg's auc: 0.745585 + 0.00555995
[804]	cv_agg's auc: 0.745583 + 0.00554931
[805]	cv_agg's auc: 0.745575 + 0.00555035
[806]	cv_agg's auc: 0.745583 + 0.00555208
[807]	cv_agg's auc: 0.745595 + 0.00556137
[808]	cv_agg's auc: 0.745593 + 0.00554329
[809]	cv_agg's auc: 0.74559 + 0.00554832
[810]	cv_agg's auc: 0.745601 + 0.00557413
[811]	cv_agg's auc: 0.745605 + 0.00560083
[812]	cv_agg's auc: 0.74561 + 0.00559171
[813]	cv_agg's auc: 0.745603 + 0.00560996
[814]	cv_agg's auc: 0.745608 + 0.00562378
[815]	cv_agg's auc: 0.745595 + 0.0056032

[988]	cv_agg's auc: 0.745988 + 0.005999
[989]	cv_agg's auc: 0.745988 + 0.00600156
[990]	cv_agg's auc: 0.745977 + 0.00600825
[991]	cv_agg's auc: 0.745984 + 0.00601074
[992]	cv_agg's auc: 0.74598 + 0.00599422
[993]	cv_agg's auc: 0.745966 + 0.00597692
[994]	cv_agg's auc: 0.745947 + 0.00597439
[995]	cv_agg's auc: 0.745948 + 0.00598477
[996]	cv_agg's auc: 0.745956 + 0.00598357
[997]	cv_agg's auc: 0.745943 + 0.00600038
[998]	cv_agg's auc: 0.745937 + 0.00599105
[999]	cv_agg's auc: 0.745942 + 0.00599322
[1000]	cv_agg's auc: 0.745934 + 0.00600792
[1001]	cv_agg's auc: 0.745924 + 0.00601317
[1002]	cv_agg's auc: 0.745939 + 0.00602209
[1003]	cv_agg's auc: 0.745953 + 0.00601902
[1004]	cv_agg's auc: 0.745967 + 0.00599604
[1005]	cv_agg's auc: 0.745995 + 0.00601085
[1006]	cv_agg's auc: 0.745997 + 0.00601294
[1007]	cv_agg's auc: 0.745995 + 0.00600929
[1008]	cv_agg's auc: 0.745999 + 0.00601675
[1009]	cv_agg's auc: 0.746014 + 0.00603136
[1010]	cv_agg's auc: 0.745996 + 0.00601886
[1011]	cv_agg's auc: 0.746

[1182]	cv_agg's auc: 0.746193 + 0.00623254
[1183]	cv_agg's auc: 0.746183 + 0.00622786
[1184]	cv_agg's auc: 0.746172 + 0.00623206
[1185]	cv_agg's auc: 0.746165 + 0.00623947
[1186]	cv_agg's auc: 0.746168 + 0.00623575
[1187]	cv_agg's auc: 0.746177 + 0.00622407
[1188]	cv_agg's auc: 0.746178 + 0.00625611
[1189]	cv_agg's auc: 0.74618 + 0.00626881
[1190]	cv_agg's auc: 0.746164 + 0.00626292
[1191]	cv_agg's auc: 0.746138 + 0.00628776
[1192]	cv_agg's auc: 0.746146 + 0.00631241
[1193]	cv_agg's auc: 0.746141 + 0.00630493
[1194]	cv_agg's auc: 0.74612 + 0.00632095
[1195]	cv_agg's auc: 0.746099 + 0.00632867
[1196]	cv_agg's auc: 0.746092 + 0.00634283
[1197]	cv_agg's auc: 0.746094 + 0.00636115
[1198]	cv_agg's auc: 0.746107 + 0.00636332
[1199]	cv_agg's auc: 0.746109 + 0.00637217
[1200]	cv_agg's auc: 0.746101 + 0.00636155
[1201]	cv_agg's auc: 0.746095 + 0.00636552
[1202]	cv_agg's auc: 0.746089 + 0.00639807
[1203]	cv_agg's auc: 0.746091 + 0.0064141
[1204]	cv_agg's auc: 0.746086 + 0.00643283
[1205]	cv_agg'

[1376]	cv_agg's auc: 0.746254 + 0.0071859
[1377]	cv_agg's auc: 0.746241 + 0.00721941
[1378]	cv_agg's auc: 0.746254 + 0.00723231
[1379]	cv_agg's auc: 0.746261 + 0.00723847
[1380]	cv_agg's auc: 0.746274 + 0.0072278
[1381]	cv_agg's auc: 0.74624 + 0.00719685
[1382]	cv_agg's auc: 0.746241 + 0.00717437
[1383]	cv_agg's auc: 0.746249 + 0.00716744
[1384]	cv_agg's auc: 0.746225 + 0.00716925
[1385]	cv_agg's auc: 0.746226 + 0.00718241
[1386]	cv_agg's auc: 0.746204 + 0.00719428
[1387]	cv_agg's auc: 0.746203 + 0.00719671
[1388]	cv_agg's auc: 0.746197 + 0.00721804
[1389]	cv_agg's auc: 0.746201 + 0.00721136
[1390]	cv_agg's auc: 0.746198 + 0.00722438
[1391]	cv_agg's auc: 0.746217 + 0.00720632
[1392]	cv_agg's auc: 0.746204 + 0.00722648
[1393]	cv_agg's auc: 0.7462 + 0.00722475
[1394]	cv_agg's auc: 0.746188 + 0.00723553
[1395]	cv_agg's auc: 0.74618 + 0.00725355
[1396]	cv_agg's auc: 0.746176 + 0.00724946
[1397]	cv_agg's auc: 0.746191 + 0.00723535
[1398]	cv_agg's auc: 0.746205 + 0.0072465
[1399]	cv_agg's au

[1568]	cv_agg's auc: 0.745887 + 0.00720198
[1569]	cv_agg's auc: 0.745873 + 0.00719262
[1570]	cv_agg's auc: 0.74588 + 0.00718211
[1571]	cv_agg's auc: 0.745891 + 0.00718183
[1572]	cv_agg's auc: 0.745905 + 0.00718613
[1573]	cv_agg's auc: 0.745891 + 0.00719224
[1574]	cv_agg's auc: 0.745885 + 0.00719287
[1575]	cv_agg's auc: 0.745883 + 0.00721192
[1576]	cv_agg's auc: 0.745902 + 0.00720081
[1577]	cv_agg's auc: 0.745911 + 0.00720597
[1578]	cv_agg's auc: 0.745923 + 0.00720034
[1579]	cv_agg's auc: 0.745909 + 0.00720963
[1580]	cv_agg's auc: 0.745905 + 0.00722736
[1581]	cv_agg's auc: 0.745924 + 0.00723599
[1582]	cv_agg's auc: 0.745915 + 0.00724342
[1583]	cv_agg's auc: 0.745907 + 0.00724199
[1584]	cv_agg's auc: 0.745884 + 0.00724304
[1585]	cv_agg's auc: 0.745871 + 0.00724131
[1586]	cv_agg's auc: 0.745878 + 0.00722258
[1587]	cv_agg's auc: 0.745893 + 0.00720353
[1588]	cv_agg's auc: 0.745884 + 0.0071985
[1589]	cv_agg's auc: 0.745883 + 0.00720013
[1590]	cv_agg's auc: 0.745874 + 0.00720861
[1591]	cv_agg

In [35]:
print(max(result['auc-mean']), len(result['auc-mean']))
print(check_train_score(parameters, lgb_train, X_train, y_train, skf, len(result['auc-mean'])))




0.7463071236731531 1350
[0.94371671 0.74630712]


In [36]:
model = lgb.train(parameters, lgb_train, num_boost_round = len(result['auc-mean']), verbose_eval=4)
                  
y_lgb3 = model.predict(X_test)

test['0'] = y_lgb3
header = ['_VAL_']
test['0'].to_csv('lgb3.csv',sep =',', header = header, index_label = ['_ID_']) 


#lb 0.76415792


# CATBOOST

In [79]:
ctb_model = ctb.CatBoostClassifier(iterations = 3066, learning_rate=0.01, eval_metric='AUC', 
                                    random_seed=42,thread_count = 4)

ctb_model.fit(X_train, y_train)
y_ctb = ctb_model.predict_proba(X_test)



0:	learn: 0.6295984	total: 450ms	remaining: 22m 57s
1:	learn: 0.6401887	total: 863ms	remaining: 22m 2s
2:	learn: 0.6532298	total: 1.26s	remaining: 21m 32s
3:	learn: 0.6550836	total: 1.72s	remaining: 21m 58s
4:	learn: 0.6666549	total: 2.15s	remaining: 21m 58s
5:	learn: 0.6676934	total: 2.49s	remaining: 21m 9s
6:	learn: 0.6706928	total: 2.79s	remaining: 20m 20s
7:	learn: 0.6723774	total: 3.1s	remaining: 19m 45s
8:	learn: 0.6733186	total: 3.48s	remaining: 19m 42s
9:	learn: 0.6755539	total: 3.89s	remaining: 19m 48s
10:	learn: 0.6779472	total: 4.19s	remaining: 19m 23s
11:	learn: 0.6771475	total: 4.42s	remaining: 18m 44s
12:	learn: 0.6770414	total: 4.71s	remaining: 18m 25s
13:	learn: 0.6788063	total: 4.95s	remaining: 17m 58s
14:	learn: 0.6794701	total: 5.16s	remaining: 17m 29s
15:	learn: 0.6798922	total: 5.4s	remaining: 17m 9s
16:	learn: 0.6825270	total: 5.67s	remaining: 16m 56s
17:	learn: 0.6843770	total: 5.97s	remaining: 16m 51s
18:	learn: 0.6845469	total: 6.25s	remaining: 16m 42s
19:	lear

155:	learn: 0.7139626	total: 50s	remaining: 15m 32s
156:	learn: 0.7142121	total: 50.2s	remaining: 15m 30s
157:	learn: 0.7145339	total: 50.5s	remaining: 15m 28s
158:	learn: 0.7146219	total: 50.7s	remaining: 15m 27s
159:	learn: 0.7146867	total: 51s	remaining: 15m 26s
160:	learn: 0.7148356	total: 51.3s	remaining: 15m 25s
161:	learn: 0.7149598	total: 51.5s	remaining: 15m 23s
162:	learn: 0.7150338	total: 51.7s	remaining: 15m 20s
163:	learn: 0.7151625	total: 51.9s	remaining: 15m 18s
164:	learn: 0.7153253	total: 52.2s	remaining: 15m 17s
165:	learn: 0.7154433	total: 52.4s	remaining: 15m 15s
166:	learn: 0.7156988	total: 52.9s	remaining: 15m 18s
167:	learn: 0.7157560	total: 53.7s	remaining: 15m 25s
168:	learn: 0.7159047	total: 54.1s	remaining: 15m 26s
169:	learn: 0.7162689	total: 54.4s	remaining: 15m 27s
170:	learn: 0.7164251	total: 54.7s	remaining: 15m 26s
171:	learn: 0.7165364	total: 55s	remaining: 15m 24s
172:	learn: 0.7167053	total: 55.3s	remaining: 15m 24s
173:	learn: 0.7168967	total: 55.6s

307:	learn: 0.7328624	total: 1m 31s	remaining: 13m 42s
308:	learn: 0.7329424	total: 1m 32s	remaining: 13m 41s
309:	learn: 0.7329687	total: 1m 32s	remaining: 13m 40s
310:	learn: 0.7330616	total: 1m 32s	remaining: 13m 40s
311:	learn: 0.7331802	total: 1m 32s	remaining: 13m 39s
312:	learn: 0.7333107	total: 1m 33s	remaining: 13m 39s
313:	learn: 0.7333464	total: 1m 33s	remaining: 13m 38s
314:	learn: 0.7334288	total: 1m 33s	remaining: 13m 37s
315:	learn: 0.7335274	total: 1m 33s	remaining: 13m 37s
316:	learn: 0.7336133	total: 1m 34s	remaining: 13m 37s
317:	learn: 0.7337160	total: 1m 34s	remaining: 13m 37s
318:	learn: 0.7337860	total: 1m 34s	remaining: 13m 36s
319:	learn: 0.7338943	total: 1m 35s	remaining: 13m 36s
320:	learn: 0.7339777	total: 1m 35s	remaining: 13m 35s
321:	learn: 0.7340238	total: 1m 35s	remaining: 13m 35s
322:	learn: 0.7341105	total: 1m 35s	remaining: 13m 33s
323:	learn: 0.7341269	total: 1m 36s	remaining: 13m 34s
324:	learn: 0.7341946	total: 1m 36s	remaining: 13m 33s
325:	learn

458:	learn: 0.7433586	total: 2m 9s	remaining: 12m 13s
459:	learn: 0.7434103	total: 2m 9s	remaining: 12m 13s
460:	learn: 0.7434464	total: 2m 9s	remaining: 12m 12s
461:	learn: 0.7434882	total: 2m 9s	remaining: 12m 12s
462:	learn: 0.7435243	total: 2m 10s	remaining: 12m 11s
463:	learn: 0.7435807	total: 2m 10s	remaining: 12m 11s
464:	learn: 0.7436606	total: 2m 10s	remaining: 12m 10s
465:	learn: 0.7437367	total: 2m 10s	remaining: 12m 10s
466:	learn: 0.7438023	total: 2m 11s	remaining: 12m 9s
467:	learn: 0.7439201	total: 2m 11s	remaining: 12m 9s
468:	learn: 0.7439829	total: 2m 11s	remaining: 12m 8s
469:	learn: 0.7440177	total: 2m 11s	remaining: 12m 8s
470:	learn: 0.7440809	total: 2m 12s	remaining: 12m 7s
471:	learn: 0.7441289	total: 2m 12s	remaining: 12m 7s
472:	learn: 0.7441482	total: 2m 12s	remaining: 12m 6s
473:	learn: 0.7442307	total: 2m 12s	remaining: 12m 6s
474:	learn: 0.7443042	total: 2m 13s	remaining: 12m 5s
475:	learn: 0.7443540	total: 2m 13s	remaining: 12m 5s
476:	learn: 0.7444030	to

608:	learn: 0.7520956	total: 2m 43s	remaining: 11m
609:	learn: 0.7521652	total: 2m 44s	remaining: 11m
610:	learn: 0.7521854	total: 2m 44s	remaining: 11m
611:	learn: 0.7522898	total: 2m 44s	remaining: 10m 59s
612:	learn: 0.7523488	total: 2m 44s	remaining: 10m 59s
613:	learn: 0.7523726	total: 2m 45s	remaining: 10m 59s
614:	learn: 0.7524540	total: 2m 45s	remaining: 10m 58s
615:	learn: 0.7524921	total: 2m 45s	remaining: 10m 58s
616:	learn: 0.7525506	total: 2m 45s	remaining: 10m 57s
617:	learn: 0.7526200	total: 2m 45s	remaining: 10m 57s
618:	learn: 0.7526683	total: 2m 46s	remaining: 10m 56s
619:	learn: 0.7527075	total: 2m 46s	remaining: 10m 56s
620:	learn: 0.7527391	total: 2m 46s	remaining: 10m 56s
621:	learn: 0.7527983	total: 2m 46s	remaining: 10m 55s
622:	learn: 0.7528600	total: 2m 47s	remaining: 10m 55s
623:	learn: 0.7528868	total: 2m 47s	remaining: 10m 55s
624:	learn: 0.7529213	total: 2m 47s	remaining: 10m 54s
625:	learn: 0.7529503	total: 2m 47s	remaining: 10m 54s
626:	learn: 0.7530022	

...

2420:	learn: 0.8269460	total: 10m 4s	remaining: 2m 41s
2421:	learn: 0.8269868	total: 10m 4s	remaining: 2m 40s
2422:	learn: 0.8270042	total: 10m 5s	remaining: 2m 40s
2423:	learn: 0.8270343	total: 10m 5s	remaining: 2m 40s
2424:	learn: 0.8270593	total: 10m 5s	remaining: 2m 40s
2425:	learn: 0.8270894	total: 10m 6s	remaining: 2m 39s
2426:	learn: 0.8271263	total: 10m 6s	remaining: 2m 39s
2427:	learn: 0.8271679	total: 10m 6s	remaining: 2m 39s
2428:	learn: 0.8272089	total: 10m 7s	remaining: 2m 39s
2429:	learn: 0.8272432	total: 10m 7s	remaining: 2m 38s
2430:	learn: 0.8272860	total: 10m 7s	remaining: 2m 38s
2431:	learn: 0.8273057	total: 10m 7s	remaining: 2m 38s
2432:	learn: 0.8273501	total: 10m 8s	remaining: 2m 38s
2433:	learn: 0.8273599	total: 10m 8s	remaining: 2m 37s
2434:	learn: 0.8273744	total: 10m 8s	remaining: 2m 37s
2435:	learn: 0.8274126	total: 10m 8s	remaining: 2m 37s
2436:	learn: 0.8274359	total: 10m 9s	remaining: 2m 37s
2437:	learn: 0.8274465	total: 10m 9s	remaining: 2m 36s
2438:	lear

2568:	learn: 0.8312844	total: 10m 41s	remaining: 2m 4s
2569:	learn: 0.8313159	total: 10m 42s	remaining: 2m 3s
2570:	learn: 0.8313247	total: 10m 42s	remaining: 2m 3s
2571:	learn: 0.8313532	total: 10m 42s	remaining: 2m 3s
2572:	learn: 0.8313900	total: 10m 42s	remaining: 2m 3s
2573:	learn: 0.8314160	total: 10m 42s	remaining: 2m 2s
2574:	learn: 0.8314375	total: 10m 43s	remaining: 2m 2s
2575:	learn: 0.8314721	total: 10m 43s	remaining: 2m 2s
2576:	learn: 0.8314978	total: 10m 43s	remaining: 2m 2s
2577:	learn: 0.8315179	total: 10m 43s	remaining: 2m 1s
2578:	learn: 0.8315417	total: 10m 44s	remaining: 2m 1s
2579:	learn: 0.8315638	total: 10m 44s	remaining: 2m 1s
2580:	learn: 0.8315883	total: 10m 44s	remaining: 2m 1s
2581:	learn: 0.8316145	total: 10m 44s	remaining: 2m
2582:	learn: 0.8316481	total: 10m 45s	remaining: 2m
2583:	learn: 0.8316724	total: 10m 45s	remaining: 2m
2584:	learn: 0.8316927	total: 10m 45s	remaining: 2m
2585:	learn: 0.8317119	total: 10m 45s	remaining: 1m 59s
2586:	learn: 0.831753

2716:	learn: 0.8354737	total: 11m 25s	remaining: 1m 28s
2717:	learn: 0.8355110	total: 11m 26s	remaining: 1m 27s
2718:	learn: 0.8355398	total: 11m 26s	remaining: 1m 27s
2719:	learn: 0.8355654	total: 11m 26s	remaining: 1m 27s
2720:	learn: 0.8355903	total: 11m 27s	remaining: 1m 27s
2721:	learn: 0.8356144	total: 11m 27s	remaining: 1m 26s
2722:	learn: 0.8356337	total: 11m 27s	remaining: 1m 26s
2723:	learn: 0.8356458	total: 11m 27s	remaining: 1m 26s
2724:	learn: 0.8356685	total: 11m 28s	remaining: 1m 26s
2725:	learn: 0.8357167	total: 11m 28s	remaining: 1m 25s
2726:	learn: 0.8357442	total: 11m 28s	remaining: 1m 25s
2727:	learn: 0.8357728	total: 11m 28s	remaining: 1m 25s
2728:	learn: 0.8357978	total: 11m 29s	remaining: 1m 25s
2729:	learn: 0.8357994	total: 11m 29s	remaining: 1m 24s
2730:	learn: 0.8358332	total: 11m 29s	remaining: 1m 24s
2731:	learn: 0.8358595	total: 11m 29s	remaining: 1m 24s
2732:	learn: 0.8358894	total: 11m 30s	remaining: 1m 24s
2733:	learn: 0.8359188	total: 11m 30s	remaining:

2865:	learn: 0.8392010	total: 12m 5s	remaining: 50.6s
2866:	learn: 0.8392553	total: 12m 5s	remaining: 50.4s
2867:	learn: 0.8392731	total: 12m 5s	remaining: 50.1s
2868:	learn: 0.8392993	total: 12m 6s	remaining: 49.9s
2869:	learn: 0.8393207	total: 12m 6s	remaining: 49.6s
2870:	learn: 0.8393585	total: 12m 6s	remaining: 49.3s
2871:	learn: 0.8393788	total: 12m 6s	remaining: 49.1s
2872:	learn: 0.8394207	total: 12m 7s	remaining: 48.8s
2873:	learn: 0.8394485	total: 12m 7s	remaining: 48.6s
2874:	learn: 0.8394706	total: 12m 7s	remaining: 48.3s
2875:	learn: 0.8395033	total: 12m 7s	remaining: 48.1s
2876:	learn: 0.8395293	total: 12m 7s	remaining: 47.8s
2877:	learn: 0.8395327	total: 12m 8s	remaining: 47.6s
2878:	learn: 0.8395610	total: 12m 8s	remaining: 47.3s
2879:	learn: 0.8395828	total: 12m 8s	remaining: 47.1s
2880:	learn: 0.8396131	total: 12m 8s	remaining: 46.8s
2881:	learn: 0.8396272	total: 12m 9s	remaining: 46.5s
2882:	learn: 0.8396550	total: 12m 9s	remaining: 46.3s
2883:	learn: 0.8396734	total

3015:	learn: 0.8429740	total: 12m 45s	remaining: 12.7s
3016:	learn: 0.8429897	total: 12m 45s	remaining: 12.4s
3017:	learn: 0.8430212	total: 12m 46s	remaining: 12.2s
3018:	learn: 0.8430401	total: 12m 46s	remaining: 11.9s
3019:	learn: 0.8430659	total: 12m 46s	remaining: 11.7s
3020:	learn: 0.8431007	total: 12m 47s	remaining: 11.4s
3021:	learn: 0.8431535	total: 12m 47s	remaining: 11.2s
3022:	learn: 0.8431922	total: 12m 47s	remaining: 10.9s
3023:	learn: 0.8432103	total: 12m 47s	remaining: 10.7s
3024:	learn: 0.8432348	total: 12m 48s	remaining: 10.4s
3025:	learn: 0.8432690	total: 12m 48s	remaining: 10.2s
3026:	learn: 0.8432985	total: 12m 48s	remaining: 9.9s
3027:	learn: 0.8433220	total: 12m 48s	remaining: 9.65s
3028:	learn: 0.8433475	total: 12m 49s	remaining: 9.39s
3029:	learn: 0.8433683	total: 12m 49s	remaining: 9.14s
3030:	learn: 0.8433827	total: 12m 49s	remaining: 8.88s
3031:	learn: 0.8433994	total: 12m 49s	remaining: 8.63s
3032:	learn: 0.8434356	total: 12m 49s	remaining: 8.38s
3033:	learn

In [82]:
test['0'] = y_ctb[:,1]
header = ['_VAL_']
test['0'].to_csv('ctb1.csv',sep =',', header = header, index_label = ['_ID_']) 



In [3]:
print('CV score: ', (0.74116 + 0.75946 + 0.7475)/3)
print('LB score: ', 0.75188722)

CV score:  0.7493733333333333
LB score:  0.75188722
